In [ ]:
# Use two packages to open Excel files
#!pip install xlrd --break-system-packages

#!pip install openpyxl --break-system-packages

# Install the disambiguation package (no need for a master list of firm names)
#pip install disamby --break-system-packages
#!pip install rapidfuzz

# Load package for data manipulation purposes
import pandas as pd
import re
from rapidfuzz import fuzz, process

In [ ]:
# Load in June 2015 data to make a complete set of data; get rid of associations that list condo in name and those with zero listed units

june15_df = pd.read_excel("../Data/NRED/2015/NRED HOA - 06 2015.xls")

june15_df = june15_df[~june15_df['Name'].str.contains('condo', case=False, na=False)]
june15_df = june15_df[~(june15_df['# of Units'] == 0)]

june15_df 

In [ ]:
# Create a subset of observations that have management companies associated with them 
# Then sort in ascending alphabetical order according to the management company names
june15_subset = june15_df[june15_df['Address1'].str.contains('C/O', na=False)].sort_values(by='Address1')
june15_subset['City'] = june15_subset['City'].str.title()
june15_subset['Address1'] = june15_subset['Address1'].str.replace(r'\bC/O\b', '', regex=True).str.title()
june15_subset['Address2'] = june15_subset['Address2'].str.title()
june15_subset['Name'] = june15_subset['Name'].str.title()

june15_subset

In [ ]:
# Group and rename common entries that are unique in name and by listed phone number
# Proactively clean so that firms are "umbrella-ed" with multiple addresses and phone numbers under them

def singularize_word(word):
    if len(word) > 3 and word.endswith('s') and not word.endswith('ss'):
        return word[:-1]
    return word

def normalize_address(address):
    if pd.isnull(address):
        return ''
    address = address.lower()
    
    # Normalize common road types
    address = re.sub(r'\b(streets|street|st)\b', 'st', address)
    address = re.sub(r'\b(roads|road|rd)\b', 'rd', address)
    address = re.sub(r'\b(avenues|avenue|ave)\b', 'ave', address)
    address = re.sub(r'\b(boulevards|boulevard|blvd)\b', 'blvd', address)
    address = re.sub(r'\b(suites|suite|ste|se)\b', 'ste', address) 
    address = re.sub(r'\b(apartments|apartment|apt)\b', 'apt', address)
    address = re.sub(r'\b(forts|fort|ft)\b', 'ft', address)
    
    # Standardize street directions
    address = re.sub(r'\b(south|s)\b', 's', address)
    address = re.sub(r'\b(north|n)\b', 'n', address)
    address = re.sub(r'\b(east|e)\b', 'e', address)
    address = re.sub(r'\b(west|w)\b', 'w', address)
    address = re.sub(r'\b(southeast|se)\b', 'se', address)
    address = re.sub(r'\b(northeast|ne)\b', 'ne', address)
    address = re.sub(r'\b(southwest|sw)\b', 'sw', address)
    address = re.sub(r'\b(northwest|w)\b', 'nw', address)
    
    # Eliminate firm office indicators
    address = re.split(r'\b(suite|ste|apt|apartment|unit|site)\b', address)[0]
    
    # Remove punctuation and number sign
    address = re.sub(r'[.,]', '', address)
    address = re.sub(r'\s+', ' ', address).strip()
    address = re.sub(r'#', '', address)

    # Singularize words
    address = ' '.join(singularize_word(word) for word in address.split())

    return address

def normalize_firm_name(name):
    if pd.isnull(name):
        return ''
    name = name.lower().strip()
    
    # Take off suffixes
    name = re.sub(r'\b(llc|inc|corp|co|ltd)\b', '', name)
    
    # Take off commas etc.
    name = re.sub(r'[^\w\s]', '', name)
    
    # Remove spaces
    name = re.sub(r'\s+', ' ', name)
    
    #Singularize
    name = ' '.join(singularize_word(word) for word in name.split())
    
    return name

def normalize_phone(phone):
    if pd.isnull(phone):
        return ''
    phone = re.sub(r'\D', '', str(phone)) 
    return phone

def get_mode(series):
    mode = series.mode()
    return mode.iloc[0] if not mode.empty else series.iloc[0]

# Normalize all of the needed fields
june15_subset['Address2_Normalized'] = june15_subset['Address2'].apply(normalize_address)
june15_subset['Firm_Normalized'] = june15_subset['Address1'].apply(normalize_firm_name)
june15_subset['Phone_Normalized'] = june15_subset['Telephone'].apply(normalize_phone)

# Detect duplicates with two separate keys
june15_subset['Phone_Key'] = june15_subset['Phone_Normalized']
june15_subset['Address_Key'] = june15_subset['Address2_Normalized']

# Canonical firm info
canonical_phone = (
    june15_subset.groupby('Phone_Key')
    .agg({'Firm_Normalized': get_mode})
    .rename(columns={'Firm_Normalized':'Firm_ByPhone'})
    .reset_index()
)

canonical_address = (
    june15_subset.groupby('Address_Key')
    .agg({'Firm_Normalized': get_mode})
    .rename(columns={'Firm_Normalized':'Firm_ByAddress'})
    .reset_index()
)

june15_subset = june15_subset.merge(canonical_phone, on='Phone_Key', how='left')
june15_subset = june15_subset.merge(canonical_address, on='Address_Key', how='left')

def pick_canonical(row): # I NEED TO UNDERSTAND WHAT IS GOING ON HERE
    if pd.notnull(row['Firm_ByPhone']):
        return row['Firm_ByPhone']
    elif pd.notnull(row['Firm_ByAddress']):
        return row['Firm_ByAddress']
    else:
        return row['Firm_Normalized']

june15_subset['Firm_Final'] = june15_subset.apply(pick_canonical, axis=1)

# Additional fuzzy matching to fix edge cases
unique_names = june15_subset['Firm_Final'].dropna().unique()
name_map = {}

for name in unique_names:
    if name in name_map:  
        continue
    matches = process.extract(name, unique_names, scorer=fuzz.token_set_ratio, limit=None)
    close_matches = [m[0] for m in matches if m[1] >= 80]  # PLAY AROUND WITH THRESHOLD
    for cm in close_matches:
        name_map[cm] = name  
        
june15_subset['Firm_Final'] = june15_subset['Firm_Final'].map(name_map).fillna(june15_subset['Firm_Final'])

june15_subset.to_csv('../Data/Cleaned files/june15_subset.csv', index = False)

june15_subset

In [ ]:
# Load in June 2025 data to make a complete set of data; get rid of associations that list condo in name and zero units cases

june25_df = pd.read_excel("../Data/NRED/2025/NRED HOA - 06 2025.xlsx")
june25_df = june25_df[~june25_df['Name'].str.contains('condo', case=False, na=False)]
june25_df = june25_df[~(june25_df['# of Units'] == 0)]

june25_df 

In [ ]:
# Create another subset
june25_subset = june25_df[june25_df['Address1'].str.contains('C/O', na=False)].sort_values(by='Address1')
june25_subset['City'] = june25_subset['City'].str.title()
june25_subset['Address1'] = june25_subset['Address1'].str.replace(r'\bC/O\b', '', regex=True).str.title()
june25_subset['Address2'] = june25_subset['Address2'].str.title()
june25_subset['Name'] = june25_subset['Name'].str.title()

june25_subset

In [ ]:
# Repeat process

june25_subset['Address2_Normalized'] = june25_subset['Address2'].apply(normalize_address)
june25_subset['Firm_Normalized'] = june25_subset['Address1'].apply(normalize_firm_name)
june25_subset['Phone_Normalized'] = june25_subset['Telephone'].apply(normalize_phone)

june25_subset['Phone_Key'] = june25_subset['Phone_Normalized']
june25_subset['Address_Key'] = june25_subset['Address2_Normalized']

canonical_phone = (
    june25_subset.groupby('Phone_Key')
    .agg({'Firm_Normalized': get_mode})
    .rename(columns={'Firm_Normalized':'Firm_ByPhone'})
    .reset_index()
)

canonical_address = (
    june25_subset.groupby('Address_Key')
    .agg({'Firm_Normalized': get_mode})
    .rename(columns={'Firm_Normalized':'Firm_ByAddress'})
    .reset_index()
)

june25_subset = june25_subset.merge(canonical_phone, on='Phone_Key', how='left')
june25_subset = june25_subset.merge(canonical_address, on='Address_Key', how='left')

june25_subset['Firm_Final'] = june25_subset.apply(pick_canonical, axis=1)

unique_names = june25_subset['Firm_Final'].dropna().unique()
name_map = {}

for name in unique_names:
    if name in name_map:  
        continue
    matches = process.extract(name, unique_names, scorer=fuzz.token_set_ratio, limit=None)
    close_matches = [m[0] for m in matches if m[1] >= 80]  
    for cm in close_matches:
        name_map[cm] = name  
        
june25_subset['Firm_Final'] = june25_subset['Firm_Final'].map(name_map).fillna(june25_subset['Firm_Final'])

june25_subset.to_csv('../Data/Cleaned files/june25_subset.csv', index = False)

june25_subset